In [1]:
# --- .NET assembly references (case matters in pythonnet) ---------------------
# Make sure TC client bin is discoverable first (TC_BIN must point to the DLL folder)
# e.g. setx TC_BIN "C:\Siemens\Teamcenter\soa_client\bin"
import os, sys
TC_BIN = os.environ.get("TC_BIN")
if TC_BIN and TC_BIN not in sys.path:
    sys.path.append(TC_BIN)

print(sys.path)


['C:\\Users\\hvanniekerk\\AppData\\Roaming\\uv\\python\\cpython-3.13.5-windows-x86_64-none\\python313.zip', 'C:\\Users\\hvanniekerk\\AppData\\Roaming\\uv\\python\\cpython-3.13.5-windows-x86_64-none\\DLLs', 'C:\\Users\\hvanniekerk\\AppData\\Roaming\\uv\\python\\cpython-3.13.5-windows-x86_64-none\\Lib', 'C:\\Users\\hvanniekerk\\AppData\\Roaming\\uv\\python\\cpython-3.13.5-windows-x86_64-none', 'c:\\Users\\hvanniekerk\\OneDrive - GVW Group, LLC\\Python\\.venv', '', 'c:\\Users\\hvanniekerk\\OneDrive - GVW Group, LLC\\Python\\.venv\\Lib\\site-packages', 'c:\\Users\\hvanniekerk\\OneDrive - GVW Group, LLC\\Python\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\hvanniekerk\\OneDrive - GVW Group, LLC\\Python\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\hvanniekerk\\OneDrive - GVW Group, LLC\\Python\\.venv\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\hvanniekerk\\TC']


In [29]:
from glob import glob
import pathlib
import os
import clr

files = glob(os.path.join(TC_BIN, "*.dll"))
for f in files:
    if (
        "FCCNetClientProxy.dll" not in f
        and "FCCNetClientProxy_32.dll" not in f
        and "TcMemNetBinding_32.dll" not in f
        and "TcServerNetBinding_32.dll" not in f
        and 'TcSoaFMS_32.dll' not in f
        and 'Teamcenter_SSO_32.dll' not in f
    ):
        print(f"Loading {f} ...")
        clr.AddReference(f)

Loading C:\Users\hvanniekerk\TC\Abr0SoaAdvancedBomRollupStrong.dll ...
Loading C:\Users\hvanniekerk\TC\Abr0SoaStrongModelAdvancedBomRollup.dll ...
Loading C:\Users\hvanniekerk\TC\Ac0SoaActiveCollaborationStrong.dll ...
Loading C:\Users\hvanniekerk\TC\Ac0SoaStrongModelActiveCollaboration.dll ...
Loading C:\Users\hvanniekerk\TC\Acp0SoaStrongModelAwControlPlan.dll ...
Loading C:\Users\hvanniekerk\TC\Ads0SoaAdsFoundationStrong.dll ...
Loading C:\Users\hvanniekerk\TC\Ads1SoaStrongModelAwAdsFoundation.dll ...
Loading C:\Users\hvanniekerk\TC\Aed0SoaStrongModelAutomationElectricDesign.dll ...
Loading C:\Users\hvanniekerk\TC\Aft0SoaStrongModelSafetyarchitectint.dll ...
Loading C:\Users\hvanniekerk\TC\Am0SoaStrongModelAccessManagerAw.dll ...
Loading C:\Users\hvanniekerk\TC\Amx0SoaAssignmentMatrixStrong.dll ...
Loading C:\Users\hvanniekerk\TC\Amx0SoaStrongModelAssignmentMatrix.dll ...
Loading C:\Users\hvanniekerk\TC\Amx1SoaStrongModelAssignmentmatrixaw.dll ...
Loading C:\Users\hvanniekerk\TC\Ap0S

In [36]:
# dm_resolver.py
from __future__ import annotations

import importlib
from typing import Any, Tuple, Optional, Iterable

# pythonnet / .NET
import System
from System import AppDomain


def _iter_types() -> Iterable[System.Type]:
    for asm in AppDomain.CurrentDomain.GetAssemblies():
        try:
            for t in asm.GetTypes():
                yield t
        except Exception:
            # Some reflection-only or dynamic assemblies may throw
            continue


def _pick_service_type() -> Optional[System.Type]:
    """Prefer Strong.* DataManagementService; else Core.*."""
    strong: list[System.Type] = []
    weak:   list[System.Type] = []
    for t in _iter_types():
        try:
            if t is None or t.Namespace is None:
                continue
            if t.Name != "DataManagementService":
                continue
            full = t.FullName or ""
            if ".Services.Strong." in full:
                strong.append(t)
            elif ".Services.Core" in full:
                weak.append(t)
        except Exception:
            continue

    # Heuristic: prefer Strong.*; if multiple, prefer ones in Cad/Core domains, in that order.
    def _score(tt: System.Type) -> tuple[int, int]:
        f = tt.FullName or ""
        # domain preference: Cad first (common for DM strong in some kits), then Core, otherwise neutral
        dom_score = 0
        if ".Strong.Cad." in f:
            dom_score = 2
        elif ".Strong.Core." in f:
            dom_score = 1
        return (1, dom_score) if ".Services.Strong." in f else (0, 0)

    if strong:
        strong.sort(key=_score, reverse=True)
        return strong[0]
    if weak:
        return weak[0]
    return None


def _candidate_dm_type_namespaces(base_ns: str) -> list[str]:
    """
    Build a candidate list like:
      base_ns + (._YYYY_MM)? + .Datamanagement or .DataManagement
    Then extend by *observed* namespaces in loaded assemblies that match base_ns.
    """
    # Usual suspects first
    common_versions = ["_2015_10","_2012_09","_2011_06","_2010_09","_2008_06","_2007_06","_2006_03",""]
    suffixes = ["Datamanagement", "DataManagement"]

    initial = []
    for ver in common_versions:
        for suf in suffixes:
            if ver:
                initial.append(f"{base_ns}.{ver}.{suf}")
            else:
                initial.append(f"{base_ns}.{suf}")

    # Now derive from *actual loaded* types
    observed = set()
    for t in _iter_types():
        ns = t.Namespace
        if not ns or not ns.startswith(base_ns + "."):
            continue
        last = ns.split(".")[-1]
        if last in ("Datamanagement", "DataManagement"):
            observed.add(ns)

    # Merge with initial, preserving order (observed first — reality beats guesses)
    ordered = list(observed) + [ns for ns in initial if ns not in observed]
    return ordered


def _import_ns(ns: str) -> Optional[Any]:
    try:
        return importlib.import_module(ns)
    except Exception:
        return None


def resolve_dm_namespace(verbose: bool = False) -> Tuple[Any, Any, str, str]:
    """
    Returns:
        (DMServiceClass, DMTypesModule, flavor, types_namespace_str)

        - DMServiceClass: the .NET class exposing static GetService(Connection)
        - DMTypesModule:  the Python module wrapper for the Datamanagement/DataManagement namespace
        - flavor:         'strong' or 'weak'
        - types_namespace_str: the fully-qualified namespace string for types
    Raises:
        ImportError if nothing suitable is found.
    """
    svc_type = _pick_service_type()
    if not svc_type:
        raise ImportError(
            "No DataManagementService type found in loaded assemblies. "
            "Check that TcSoa* and TcServices* DLLs are actually loaded."
        )

    base_ns = svc_type.Namespace  # e.g., Teamcenter.Services.Strong.Cad  or Teamcenter.Services.Core
    flavor = "strong" if ".Services.Strong." in base_ns else "weak"

    if verbose:
        print(f"[dm] service: {svc_type.FullName}  (base_ns={base_ns}, flavor={flavor})")

    # 1) Try to locate types under the same domain (base_ns)
    tried: list[str] = []
    for cand in _candidate_dm_type_namespaces(base_ns):
        tried.append(cand)
        mod = _import_ns(cand)
        if mod is not None:
            if verbose:
                print(f"[dm] types: {cand}")
            return svc_type, mod, flavor, cand

    # 2) If that failed (e.g., service lives in Strong.Cad but types landed in Strong.Core),
    #    scan all loaded namespaces for *any* Strong.* Datamanagement/DataManagement.
    global_candidates = set()
    for t in _iter_types():
        ns = t.Namespace
        if not ns:
            continue
        if ".Services.Strong." in ns:
            last = ns.split(".")[-1]
            if last in ("Datamanagement", "DataManagement"):
                global_candidates.add(ns)

    for cand in sorted(global_candidates):
        tried.append(cand)
        mod = _import_ns(cand)
        if mod is not None:
            if verbose:
                print(f"[dm] types (cross-domain): {cand}")
            return svc_type, mod, "strong", cand

    # 3) Fall back to weak-typed Core if available
    try:
        import Teamcenter.Services.Core as Core
        mod = _import_ns("Teamcenter.Services.Core._2006_03.DataManagement")
        if mod is not None:
            if verbose:
                print("[dm] falling back to weak-typed Core._2006_03.DataManagement")
            return Core.DataManagementService, mod, "weak", "Teamcenter.Services.Core._2006_03.DataManagement"
    except Exception:
        pass

    msg = [
        f"Found service type {svc_type.FullName}, but could not resolve its Datamanagement/DataManagement types.",
        "Namespaces tried (in order):",
        *("  - " + s for s in tried),
        "Make sure your TC client kit with strong proxies is fully loaded and note that some kits place",
        "DataManagement types under different domains (e.g., Strong.Cad vs Strong.Core) or different version folders."
    ]
    raise ImportError("\n".join(msg))


def get_service_factory(DMServiceClass: Any):
    """Return a small factory that does DMServiceClass.GetService(conn)."""
    def _factory(conn: Any) -> Any:
        return DMServiceClass.GetService(conn)
    return _factory


def list_available_dm_namespaces() -> list[str]:
    """Utility to help you see what the kit exposes."""
    ns = set()
    for t in _iter_types():
        n = t.Namespace
        if not n:
            continue
        tail = n.split(".")[-1]
        if tail in ("Datamanagement", "DataManagement"):
            ns.add(n)
    return sorted(ns)


In [50]:
for t in _iter_types():
    ns = t.Namespace
    if ns is not None and ns.startswith("Teamcenter"):
        print(t.FullName)

Teamcenter.Soa.SoaConstants
Teamcenter.Soa.Internal.Common.AnyCredentials
Teamcenter.Soa.Internal.Common.NoDTDResolver
Teamcenter.Soa.Internal.Common.AuthUtils
Teamcenter.Soa.Internal.Common.Monitor
Teamcenter.Soa.Internal.Common.PolicyMarshaller
Teamcenter.Soa.Exceptions.CanceledOperationException
Teamcenter.Soa.Exceptions.ExceptionMapper
Teamcenter.Soa.Exceptions.NotLoadedException
Teamcenter.Soa.Common.ObjectPropertyPolicy
Teamcenter.Soa.Common.PolicyProperty
Teamcenter.Soa.Common.PolicyType
Teamcenter.Soa.Common.Tpc
Teamcenter.Soa.Common.TpcPackage
Teamcenter.Soa.Common.Version
Teamcenter.Soa.Common.Utils.XmlBindingUtils
Teamcenter.Schemas.Soa.Objectpropertypolicy.ObjectPropertyPolicy
Teamcenter.Schemas.Soa.Objectpropertypolicy.ObjectType
Teamcenter.Schemas.Soa.Objectpropertypolicy.Property
Teamcenter.Schemas.Soa._2006_09.Clientcontext.NameBooleanValue
Teamcenter.Schemas.Soa._2006_09.Clientcontext.NameIntegerValue
Teamcenter.Schemas.Soa._2006_09.Clientcontext.NameStringValue
Teamce

In [39]:
# 1) Load DLLs first (your glob loader is fine).
# 2) Then:
# from dm_resolver import resolve_dm_namespace, get_service_factory, list_available_dm_namespaces

# Optional: see what Datamanagement namespaces actually exist:
for n in list_available_dm_namespaces():
    print(" •", n)

# Resolve (with debug prints):
DMService, DMTypes, DM_FLAVOR, DM_TYPES_NS = resolve_dm_namespace(verbose=True)
getService = get_service_factory(DMService)

print("\nSummary:")
print("  flavor     :", DM_FLAVOR)
print("  service    :", DMService)
print("  types ns   :", DM_TYPES_NS)


 • Abr0.Services.Strong.Advancedbomrollup._2023_12.DataManagement
 • Abr0.Services.Strong.Advancedbomrollup._2024_06.DataManagement
 • Awb0.Services.Internal.Strong.Activeworkspacebom._2019_12.DataManagement
 • Awn0.Services.Internal.Strong.Nx._2020_05.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2012_10.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2013_12.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2015_10.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2016_03.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2016_04.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2016_12.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2017_06.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2017_12.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2018_05.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2019_06.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2020_05.DataManagement
 • Awp0.Services.Internal.Strong.Aws2._2020_12.Dat

In [41]:
dir(DMTypes)

['CadAttrMappingDefinitionInfo',
 'DataManagement',
 'DatasetInfo3',
 'MappedDatasetAttrPropertyInfo',
 'PartInfo3',
 'ResolveAttrMappingsInfo',
 'ResolveAttrMappingsResponse',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__file__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__loader__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__package__',
 '__path__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__spec__',
 '__str__',
 '__subclasshook__']

In [38]:
# list a few names in the resolved DM types namespace
names = [n for n in dir(DMTypes) if "GetItem" in n or "Related" in n]
print(names[:50])


[]


In [33]:
# dm_resolver.py
from __future__ import annotations
from typing import Any, Optional, Tuple
import sys

import clr  # already loaded
from System import Array
from System import String
from System import Type
from System import Version
from System import AppDomain

# Utility: call getService/GetService without caring about casing
def _get_service_factory(dm_service_type) -> Any:
    for name in ("getService", "GetService"):
        if hasattr(dm_service_type, name):
            return getattr(dm_service_type, name)
    raise AttributeError(
        f"{dm_service_type.FullName} exposes neither getService nor GetService."
    )

def _import_ns(ns: str) -> Optional[Any]:
    try:
        # __import__ with fromlist ensures we actually load the tail module
        return __import__(ns, fromlist=['*'])
    except Exception:
        return None

def resolve_dm_namespace() -> Tuple[Any, Any, str]:
    """
    Locate Teamcenter's DataManagement service + its request/response types.

    Returns:
        (DMServiceClass, DMTypesNamespaceOrStr, 'strong'|'weak')

    DMTypesNamespaceOrStr is either a Python module (preferred) or a namespace string
    (when we couldn't import the module but we still know the namespace).
    """
    # 1) Find the service type by reflection across all loaded assemblies
    asms = list(AppDomain.CurrentDomain.GetAssemblies())
    candidates = []
    for asm in asms:
        try:
            for t in asm.GetTypes():
                # We only care about DataManagementService under Teamcenter.Services.*
                if (t is not None and t.Name == "DataManagementService"
                        and t.Namespace is not None
                        and t.Namespace.startswith("Teamcenter.Services")):
                    candidates.append(t)
        except Exception:
            # Dynamic/ref emit assemblies can throw on GetTypes(); ignore
            continue

    if not candidates:
        raise ImportError(
            "Could not locate DataManagementService in any loaded assembly. "
            "Double-check that TcSoaCoreStrong.dll and/or TcSoaCommon.dll + TcSoaClient.dll are loaded, "
            "and that no 32-bit DLLs are being mixed into a 64-bit process."
        )

    # Prefer Strong over Core (weak-typed)
    strong = [t for t in candidates if ".Strong." in t.Namespace]
    dm_service_type = (strong or candidates)[0]
    flavor = "strong" if ".Strong." in dm_service_type.Namespace else "weak"

    # 2) Figure out the companion types namespace
    # Example: Teamcenter.Services.Strong.Core._2008_06.DataManagementService  ->
    #          types under Teamcenter.Services.Strong.Core._2008_06.Datamanagement (or DataManagement)
    base_ns = dm_service_type.Namespace
    dm_types_mod = None
    dm_types_ns = None

    for suffix in ("Datamanagement", "DataManagement"):
        probe = f"{base_ns}.{suffix}"
        dm_types_mod = _import_ns(probe)
        if dm_types_mod is not None:
            dm_types_ns = probe
            break

    # If still not found, try a rep type (works even when types are in a sibling assembly)
    if dm_types_mod is None:
        rep_names = (
            "GetItemFromIdInput",
            "GetItemAndRelatedObjectsInput",
            "GetItemResponse",
            "GetItemAndRelatedObjectsResponse",
        )
        best_hit = None
        for asm in asms:
            try:
                for t in asm.GetTypes():
                    if t is None or t.Namespace is None:
                        continue
                    if not t.Namespace.startswith(base_ns):
                        continue
                    if t.Name in rep_names:
                        best_hit = t
                        break
                if best_hit:
                    break
            except Exception:
                continue
        if best_hit:
            dm_types_ns = best_hit.Namespace
            dm_types_mod = _import_ns(dm_types_ns)

    if dm_types_ns is None:
        raise ImportError(
            f"Found service type {dm_service_type.FullName}, but could not resolve its types "
            f"(e.g., {base_ns}.Datamanagement / {base_ns}.DataManagement). "
            "Check your client kit; some highly customized kits rename/relocate the types."
        )

    # 3) Return the class (not an instance), plus the module/namespace
    return dm_service_type, (dm_types_mod or dm_types_ns), flavor


# Quick self-test (optional): prints the resolved bits when run standalone
# DMService, DMTypes, FLAVOR = resolve_dm_namespace()
# print(f"[dm] flavor={FLAVOR} service={DMService.FullName} types={(getattr(DMTypes, '__name__', DMTypes))}")


In [35]:
# After your dll-loading block:
# from dm_resolver import resolve_dm_namespace, _get_service_factory

DMService, DMTypes, DM_FLAVOR = resolve_dm_namespace()
getService = _get_service_factory(DMService)

# later, once you have a Connection conn:
dm = getService(conn)   # works whether the method is getService or GetService

# DMTypes is either a module with classes OR a namespace string.
# If it’s a string, construct types by full name via reflection:
from System import Activator
from System import Type as NetType

def _new(fullname: str):
    t = NetType.GetType(fullname, throwOnError=True)
    return Activator.CreateInstance(t)

# Example: create GetItemFromIdInput no matter where it lives
try:
    # Prefer direct class access when we have a real module
    GetItemFromIdInput = getattr(DMTypes, "GetItemFromIdInput")
    inp = GetItemFromIdInput()
except Exception:
    # Fallback to reflection with the resolved namespace string
    ns = DMTypes if isinstance(DMTypes, str) else DMTypes.__name__
    inp = _new(f"{ns}.GetItemFromIdInput")


ImportError: Found service type Teamcenter.Services.Strong.Cad.DataManagementService, but could not resolve its types (e.g., Teamcenter.Services.Strong.Cad.Datamanagement / Teamcenter.Services.Strong.Cad.DataManagement). Check your client kit; some highly customized kits rename/relocate the types.

In [8]:
import clr  # pythonnet

# These are the usual assemblies needed for SOA + services + file mgmt
for asm in (
    "TcSoaCommon",
    "TcSoaClient",
    "TcServicesStrongCore",
    "TcServicesCore",  # weak-typed fallback
):
    try:
        clr.AddReference(asm)
    except Exception:
        # Some sites don't ship all of these; we'll probe dynamically below.
        pass

# Base client types (use .NET casing)
from Teamcenter.Soa.Client import Connection, FileManagementUtility, DefaultExceptionHandler  # type: ignore
# ResponseExceptionHandler is not in all client kits; don't import it unless you truly need it.


In [30]:
# -----------------------------------------------------------------------------
# Robust resolver for DataManagement service + its Datamanagement types
# -----------------------------------------------------------------------------
from typing import Any, Tuple

def resolve_dm_namespace() -> Tuple[Any, Any, str]:
    """
    Try to resolve a DataManagement service in this priority:
      1) Strong.Core (unversioned)
      2) Strong.Core._2008_06
      3) Strong.Core._2007_06
      4) Strong.Core._2006_03
      5) Core (weak-typed, unversioned service + _2006_03 Datamanagement types)

    Returns:
        (DMServiceClass, DMTypesNamespace, 'strong'|'weak')
    Raises:
        ImportError if none can be found.
    """
    # 1) Strong, unversioned (service sits directly under Strong.Core in some kits)
    try:
        import Teamcenter.Services.Strong.Core as StrongCore
        DMService = StrongCore.DataManagementService  # type: ignore[attr-defined]
        # Companion Datamanagement types are typically versioned even if service is not
        try:
            from Teamcenter.Services.Strong.Core._2008_06 import Datamanagement as DMTypes  # type: ignore
        except Exception:
            # Fall back to older type packages if needed
            for ver in ("_2007_06", "_2006_03"):
                try:
                    DMTypes = __import__(f"Teamcenter.Services.Strong.Core.{ver}", fromlist=["Datamanagement"]).Datamanagement  # type: ignore
                    break
                except Exception:
                    DMTypes = None
            if DMTypes is None:
                raise ImportError("Strong.Core Datamanagement types not found")
        return DMService, DMTypes, "strong"
    except Exception:
        pass

    # 2…4) Strong, versioned service (most common is _2008_06)
    for ver in ("_2008_06", "_2007_06", "_2006_03"):
        try:
            ver_ns = __import__(f"Teamcenter.Services.Strong.Core.{ver}", fromlist=["DataManagementService", "Datamanagement"])
            DMService = getattr(ver_ns, "DataManagementService")
            DMTypes = getattr(ver_ns, "Datamanagement")
            return DMService, DMTypes, "strong"
        except Exception:
            continue

    # 5) Weak-typed Core fallback (almost always available)
    try:
        import Teamcenter.Services.Core as Core
        from Teamcenter.Services.Core._2006_03 import DataManagement as DMTypes  # inputs/outputs live here
        DMService = Core.DataManagementService  # unversioned weak-typed service
        return DMService, DMTypes, "weak"
    except Exception:
        pass

    raise ImportError(
        "Could not resolve any DataManagement service. "
        "Ensure your TC client kit includes either Teamcenter.Services.Strong.Core or Teamcenter.Services.Core, "
        "and that TC_BIN points to the folder with those DLLs."
    )

# Example: bind once at import time
DMService, DMTypes, DM_FLAVOR = resolve_dm_namespace()
print(f"[dm] using {DM_FLAVOR} typed DataManagement: {DMService} with types {DMTypes}")


ImportError: Could not resolve any DataManagement service. Ensure your TC client kit includes either Teamcenter.Services.Strong.Core or Teamcenter.Services.Core, and that TC_BIN points to the folder with those DLLs.